# **BRAIN TUMOR DETECTION**

In [ ]:
# Setting up tensorflow GPU for Performance Gain. Reduces the amount of time taken in training with large datasets.
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


**DATSET:**

The dataset contains 7022 images of human brain MRIs which are classified into 4 classes:

1.   Glioma
2.   Meningioma
3.   No tumor
4.   Pituitary




In [ ]:
#for uploading dateset into colab environment from kaggle 
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d 'masoudnickparvar/brain-tumor-mri-dataset'

!mkdir tumor

! unzip brain-tumor-mri-dataset.zip -d tumor

Streaming output truncated to the last 5000 lines.
  inflating: tumor/Training/glioma/Tr-gl_0712.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0713.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0714.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0715.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0716.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0717.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0718.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0719.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0720.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0721.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0722.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0723.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0724.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0725.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0726.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0727.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0728.jpg  
  inflating: tumor/Training/glioma/Tr-gl_0729.jpg  
  inflating: 

In [ ]:
# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import random
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,Dropout,Input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.metrics import AUC,Precision,Recall
from tensorflow.keras.metrics import AUC, Precision, Recall, FalseNegatives, FalsePositives,TrueNegatives,TruePositives, CategoricalCrossentropy
from sklearn.metrics import confusion_matrix

import numpy as np
# from fcmeans import FCM
from matplotlib import pyplot as plt
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import os
import shutil

In [ ]:
# the different classes of the MRI images
typet=['glioma','meningioma','notumor','pituitary']
for t in typet:
  path='/content/tumor/Testing/'+t
  img_list=os.listdir(path)
  print(t)
  print(len(img_list))

  #testing meningioma
  c=0
  for i in img_list:
    img_path=path+'/'+str(i)
    img=cv2.imread(img_path)
    # converting all the images to grayscale. why?
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # fastNlMeansDenoising: Perform image denoising using Non-local Means Denoising algorithm.
    # fastNlMeansDenoising (Input, Output, float parameterRegulatingFilterStrength=3, int templateWindowSize=7, int searchWindowSize=21)
    dst1 = cv2.fastNlMeansDenoising(img, None, 3, 7, 21)
    # Contrast Limited Adaptive Histogram Equalization 
    clahe = cv2.createCLAHE(clipLimit = 5)
    img = clahe.apply(dst1)+30
    cv2.imwrite(str(i),img)



    '''imgshape=img.shape
    imgre=img.reshape((img.shape[0]*img.shape[1],3))
    fcm = FCM(n_clusters=3)
    fcm.fit(imgre)
    fcm_labels = fcm.predict(imgre)
    image=fcm_labels.reshape((imgshape[0],imgshape[1]))
    
    cv2.imwrite(str(i),image)'''
    c=c+1
    print(c) 
#removing files in images folder
  fileinimages=os.listdir(path)
  for i in fileinimages:
    os.remove(path+'/'+str(i))
    filelist=os.listdir('/content/')  
    filelistjpg=[]
  for fichier in filelist:
    if (fichier.endswith(".JPG")) or fichier.endswith('.png') or fichier.endswith('.jpeg') or fichier.endswith('.jpg'):
      filelistjpg.append(fichier)
  print(len(filelistjpg))
  for i in filelistjpg:
    source='/content/'+str(i)
    dest='/content/tumor/Testing/'+t
    shutil.move(source,dest)

glioma
300
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
2

In [ ]:
typet=['glioma','meningioma','notumor','pituitary']
for t in typet:
  path='/content/tumor/Training/'+t
  img_list=os.listdir(path)
  print(t)
  print(len(img_list))

  #testing meningioma
  c=0
  for i in img_list:
    img_path=path+'/'+str(i)
    img=cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dst1 = cv2.fastNlMeansDenoising(img,None,3,7,21)
    clahe = cv2.createCLAHE(clipLimit = 5)
    img = clahe.apply(dst1)+30
    cv2.imwrite(str(i),img)
    
    '''imgshape=img.shape
    imgre=img.reshape((img.shape[0]*img.shape[1],3))
    fcm = FCM(n_clusters=3)
    fcm.fit(imgre)
    fcm_labels = fcm.predict(imgre)
    image=fcm_labels.reshape((imgshape[0],imgshape[1]))
    
    cv2.imwrite(str(i),image)'''

    c=c+1
    print(c) 
#removing files in images folder
  fileinimages=os.listdir(path)
  for i in fileinimages:
    os.remove(path+'/'+str(i))
    filelist=os.listdir('/content/')  
    filelistjpg=[]
  for fichier in filelist:
    if (fichier.endswith(".JPG")) or fichier.endswith('.png') or fichier.endswith('.jpeg') or fichier.endswith('.jpg'):
      filelistjpg.append(fichier)
  print(len(filelistjpg))
  for i in filelistjpg:
    source='/content/'+str(i)
    dest='/content/tumor/Training/'+t
    shutil.move(source,dest)

Streaming output truncated to the last 5000 lines.
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
9

In [ ]:
# training and testing data
trainpath='/content/tumor/Training'
testpath='/content/tumor/Testing'

In [ ]:
classmode='categorical'
Batch_size=16
# ImageDataGenerator: augments images in real-time while the model is still in training.
# Augmenting images: artificially expand the size of a training dataset by creating modified versions of images in the dataset.
Train_batches = ImageDataGenerator(rescale=1./223).flow_from_directory(directory = trainpath, target_size = (224,224), batch_size = Batch_size, class_mode = classmode)
Test_batches = ImageDataGenerator(rescale=1./223).flow_from_directory(directory = testpath, target_size = (224,224), batch_size = Batch_size, class_mode = classmode)

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


**RetinaNet** is one of the best one-stage object detection models that has proven to work well with dense and small scale objects. 

RetinaNet has been formed by making two improvements over existing single stage object detection models - 
1.   **Feature Pyramid Networks (FPN)**
*    Featurized image pyramids are feature pyramids built upon image pyramids. 
*   So this would mean we take an image and subsample it into lower resolutions and smaller size images (thus, forming a pyramid).
*   FPN creates an architecture with rich semantics at all levels as it combines low-resolution semantically strong features with high-resolution semantically weak features. 

2.   **Focal Loss**
*   Focal Loss (FL) is an enhancement over Cross-Entropy Loss (CE) and is introduced to handle the class imbalance problem with single-stage object detection models. 
*   Easy examples (detections with high probabilities) result in very small loss values but collectively, they can overwhelm the model. Focal Loss reduces the loss contribution from easy examples and increases the importance of correcting missclassified examples.




In [ ]:
!pip install keras_retinanet

     |████████████████████████████████| 71 kB 5.4 MB/s 
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp37-cp37m-linux_x86_64.whl size=163916 sha256=abd054f0e32eaa906fcbf24952df084da742cb753b6a1abfc22554d7dc45acfe
  Stored in directory: /root/.cache/pip/wheels/55/a1/41/f2dbc08b26549fe00041b9bc25c299f262b0311ad68cc52522
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=692ab7d2f4b3816af795180e4c5f8de164d6e8fab5673d43550a2ee26a572b13
  Stored in directory: /root/.cache/pip/wheels/bd/ef/06/5d65f696360436c3a423020c4b7fd8c558c09ef264a0e6c575
Successfully built keras-retinanet keras-resnet


In [ ]:
# several libraries here are already imported. 
# cant we put all the imported libraries together or put them in groups as and when they r required without repeatition?
import tensorflow 
from tensorflow import keras
import numpy as np
import cv2
import os
import random
import shutil
import pandas as pd
import csv
import zipfile
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dropout, Flatten, Dense,Input
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import RandomNormal
import tensorflow.keras.backend as k
from sklearn.utils import shuffle
import io
from PIL import Image as pil_image
from keras_retinanet import layers
import tensorflow.keras.backend as k
import keras_retinanet

In [ ]:
for i in os.listdir('/content/models'):
  os.remove('/content/models/'+i)

FileNotFoundError: ignored

**ResNet (Residual Network)**
*  It's a classic neural network used as a backbone for many computer vision tasks.
*   Allows to train extremely deep neural networks with 150+ layers successfully. 
*   List item
*   List item





In [ ]:
layer_names

[-2, -51, -102]

In [ ]:
fold_num=1
# Clear tensorflow.keras backend as we're creating many models in a loop, so the global state would consume an increasing amount of memory over time
k.clear_session() 
try:
  # Create a folder for saving the trained networks
  os.mkdir('models1') 
except:
  pass
full_name='ResNet50V2-FPN-fold{}'.format(fold_num)

classes_number=4 
input_tensor=Input(shape=(224,224,3))

# Loading ResNet50V2 ImageNet pre-trained weights
weight_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False) 
# Saving the weights
weight_model.save_weights('weights.h5') 
# Loading the ResNet50V2 model without weights
base_model = tf.keras.applications.MobileNetV2(weights=None, include_top=False, input_tensor=input_tensor) 
# Loading the ImageNet weights on the ResNet50V2 model except for the first layer (because in our case, the first layer has one channel)
base_model.load_weights('weights.h5',skip_mismatch=True, by_name=True) 
  
# Create Feature Pyramid Network (FPN)
# Setting the feature channels of the FPN
feature_size= 512 
x=int(len(base_model.layers)/3)
layer_names = [-2,-1*x, -2*x,] #Layers of ResNet50V2 with different scale features  
layer_outputs = [base_model.layers[name].output for name in layer_names]
# Features of different scales, extracted from ResNet50V2
C3, C4, C5=layer_outputs 
P5           = tensorflow.keras.layers.Conv2D(feature_size, kernel_size=1, strides=1, padding='same', name='C5_reduced')(C5)
P5_upsampled = layers.UpsampleLike(name='P5_upsampled')([P5, C4])
P5           = tensorflow.keras.layers.Conv2D(feature_size, kernel_size=3, strides=1, padding='same', name='P5')(P5)
 
# Concatenate P5 elementwise to C4
P4           = tensorflow.keras.layers.Conv2D(feature_size, kernel_size=1, strides=1, padding='same', name='C4_reduced')(C4)
P4           = tensorflow.keras.layers.Concatenate(axis=3)([P5_upsampled, P4])
P4_upsampled = layers.UpsampleLike(name='P4_upsampled')([P4, C3])
P4           = tensorflow.keras.layers.Conv2D(feature_size, kernel_size=3, strides=1, name='P4')(P4)
 
# Concatenate P4 elementwise to C3
P3 = tensorflow.keras.layers.Conv2D(feature_size, kernel_size=1, strides=1, padding='same', name='C3_reduced')(C3)
P3 = tensorflow.keras.layers.Concatenate(axis=3)([P4_upsampled, P3])
P3 = tensorflow.keras.layers.Conv2D(feature_size, kernel_size=3, strides=1, name='P3')(P3)
 
# "P6 is obtained via a 3x3 stride-2 conv on C5"
P6 = tensorflow.keras.layers.Conv2D(feature_size, kernel_size=3, strides=2, padding='same', name='P6')(C5)
 
# "P7 is computed by applying ReLU followed by a 3x3 stride-2 conv on P6"
P7 = tensorflow.keras.layers.Activation('relu', name='C6_relu')(P6)
P7 = tensorflow.keras.layers.Conv2D(feature_size, kernel_size=3, strides=2, padding='same', name='P7')(P7)
 
# Run classification for each of the generated features from the pyramid
feature1 = Flatten()(P3)
dp1 = Dropout(0.5)(feature1)
# Initializers define the way to set the initial random weights of Keras layers.
preds1 = Dense(4, activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp1)
feature2 = Flatten()(P4)
dp2 = Dropout(0.5)(feature2)
preds2 = Dense(4, activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp2)
#################################################################
feature3 = Flatten()(P5)
dp3= Dropout(0.5)(feature3)
preds3 = Dense(4, activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp3)
#################################################################
feature4 = Flatten()(P6)
dp4 = Dropout(0.5)(feature4)
preds4 = Dense(4, activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp4)
#################################################################
feature5 = Flatten()(P7)
dp5 = Dropout(0.5)(feature5)
preds5 = Dense(4, activation='relu',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp5)
#################################################################
concat=tensorflow.keras.layers.Concatenate(axis=1)([preds1,preds2,preds3,preds4,preds5]) #Concatenate the predictions(Classification results) of each of the pyramid features 
out=tensorflow.keras.layers.Dense(4,activation='softmax',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(concat) #Final Classification
 
model = Model(inputs=base_model.input, outputs=out) #Create the Training Model
#######################################################
for layer in model.layers:
  layer.trainable = True
model.compile(optimizer=optimizers.Nadam(lr=0.00001), loss='categorical_crossentropy',metrics=['accuracy',AUC(),Precision(),Recall(),FalsePositives(),TrueNegatives(),TruePositives(),FalseNegatives()])
filepath="models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%full_name  # Path to save the trained models
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max') #creating checkpoint to save the best validation accuracy
callbacks_list = [checkpoint]
 
#model.fit(train_batches, epochs=20,validation_data=val_batches,shuffle=True,callbacks=callbacks_list) #start training
hist = model.fit(Train_batches, validation_data=Test_batches , epochs=30,callbacks=callbacks_list)

9420800/9406464 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


Epoch 1/30
357/357 [==============================] - ETA: 0s - loss: 0.7673 - accuracy: 0.7454 - auc: 0.9187 - precision: 0.8934 - recall: 0.5046 - false_positives: 344.0000 - true_negatives: 16792.0000 - true_positives: 2882.0000 - false_negatives: 2830.0000

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


357/357 [==============================] - 90s 164ms/step - loss: 0.7673 - accuracy: 0.7454 - auc: 0.9187 - precision: 0.8934 - recall: 0.5046 - false_positives: 344.0000 - true_negatives: 16792.0000 - true_positives: 2882.0000 - false_negatives: 2830.0000 - val_loss: 1.6134 - val_accuracy: 0.5400 - val_auc: 0.7854 - val_precision: 0.5797 - val_recall: 0.5217 - val_false_positives: 496.0000 - val_true_negatives: 3437.0000 - val_true_positives: 684.0000 - val_false_negatives: 627.0000
Epoch 2/30
357/357 [==============================] - 57s 159ms/step - loss: 0.2916 - accuracy: 0.8992 - auc: 0.9849 - precision: 0.9084 - recall: 0.8859 - false_positives: 510.0000 - true_negatives: 16626.0000 - true_positives: 5060.0000 - false_negatives: 652.0000 - val_loss: 1.5556 - val_accuracy: 0.5797 - val_auc: 0.8217 - val_precision: 0.6176 - val_recall: 0.5568 - val_false_positives: 452.0000 - val_true_negatives: 3481.0000 - val_true_positives: 730.0000 - val_false_negatives: 581.0000
Epoch 3/30
3

In [ ]:
model.load_weights('/content/models/ResNet50V2-FPN-fold1-23-0.9733.hdf5')

In [ ]:
model_metrics = model.evaluate(Test_batches,verbose=0)

f1_score = 2*( (model_metrics[3]*model_metrics[4]) / (model_metrics[3]+model_metrics[4]) )

print(f"Accuracy on test set: {round(model_metrics[1]*100,2)}%")
print(f"ROC(Receiver Operation Characteristic) AUC(Area Under Curve): {model_metrics[2]}")
print(f"Precision: {round(model_metrics[3]*100,2)}%")
print(f"Recall: {round(model_metrics[4]*100,2)}%")
print(f"F1-score: {f1_score}")
print(f"Specificity: {(model_metrics[6])/(model_metrics[6]+model_metrics[5])}")

Accuracy on test set: 97.33%
ROC(Receiver Operation Characteristic) AUC(Area Under Curve): 0.9926380515098572
Precision: 97.33%
Recall: 97.33%
F1-score: 0.9733028411865234
Specificity: 0.9911009407576913


In [ ]:
input_tensor=Input(shape=(224,224,3))
basemodel=InceptionV3(weights='imagenet', include_top=False, input_tensor=input_tensor)
x = Flatten()(basemodel.output)
x=Dense(units=256, activation='relu')(x)
x=Dense(units=512, activation='relu')(x)
x=Dropout(0.1)(x)
output = Dense(units=4, activation="softmax")(x)
basemodel = Model(inputs = basemodel.input, outputs = output)
basemodel.summary()

In [ ]:
initial_lrate = 0.0001
def decay(epoch, steps=10):
    initial_lrate = 0.0001
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
opt = RMSprop(lr=initial_lrate)
lr_sc = LearningRateScheduler(decay, verbose=1)
basemodel.compile(optimizer=opt,
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy',AUC(),Precision(),Recall(),FalsePositives(),TrueNegatives(),TruePositives(),FalseNegatives()])

In [ ]:
checkpoint = ModelCheckpoint("with_chmod_100_spatial.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, verbose=1, mode='auto',patience=3)
callback = [checkpoint]

In [ ]:
hist = basemodel.fit(Train_batches,validation_data=Test_batches,batch_size=32, epochs=30,callbacks=callback)

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")

plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):

    """

    This function prints and plots the confusion matrix.

    Normalization can be applied by setting `normalize=True`.

    """

    plt.figure(figsize=(10,10))



    plt.imshow(cm, interpolation='nearest', cmap=cmap)

    plt.title(title)

    plt.colorbar()



    tick_marks = np.arange(len(classes))

    plt.xticks(tick_marks, classes, rotation=45)

    plt.yticks(tick_marks, classes)



    if normalize:

        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        cm = np.around(cm, decimals=2)

        cm[np.isnan(cm)] = 0.0

        print("Normalized confusion matrix")

    else:

        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):

        plt.text(j, i, cm[i, j],

                 horizontalalignment="center",

                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()

    plt.ylabel('True label')

    plt.xlabel('Predicted label')

In [ ]:
target_names = []

for key in Train_batches.class_indices:

    target_names.append(key)

In [ ]:
import numpy as np

Y_pred = basemodel.predict_generator(Test_batches)

y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')

cm = confusion_matrix(Test_batches.classes, y_pred)

plot_confusion_matrix(cm, target_names, title='Confusion Matrix')

**References:**
*   https://github.com/fizyr/keras-retinanet/blob/master/keras_retinanet/models/retinanet.py
*   List item

